<a href="https://colab.research.google.com/github/fadh1l/DR-pred-IDRiD/blob/main/swin_transformer(base%20implementation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output
import pandas as pd
import os
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/MyDrive/Colab Notebooks/archive.zip"
!unzip "/content/gdrive/MyDrive/Colab Notebooks/archive2.zip"
clear_output()

In [4]:

details = pd.read_csv('/content/Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv')

In [6]:


csv_path = "/content/Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv"
image_folder = "/content/Disease Grading/1. Original Images/a. Training Set"
severity_folders = {
    0: "/content/Renamed Images/No_DR",
    1: "/content/Renamed Images/Mild",
    2: "/content/Renamed Images/Moderate",
    3: "/content/Renamed Images/Severe",
    4: "/content/Renamed Images/Proliferative_DR"
}  # Adjust names and numbers as needed

# Create folders if they don't exist
for folder_path in severity_folders.values():
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

df = pd.read_csv(csv_path)
for idx, row in df.iterrows():
    try:
        og_img_name = f"{row['Image name']}.jpg"
        sev = int(row['Retinopathy grade'])
        if sev not in severity_folders:
            print(f"Warning: Skipping image {og_img_name} due to invalid severity {sev}")
            continue

        new_img_name = f'img{idx}_{sev}.jpg'
        og_img_path = os.path.join(image_folder, og_img_name)
        new_img_path = os.path.join(severity_folders[sev], new_img_name)
        shutil.copy(og_img_path, new_img_path)
    except Exception as e:
        print(f"Error processing image {og_img_name}: {e}")  # Log or handle errors based on preferences

print("Image organization complete!")


Image organization complete!


In [7]:
pip install wandb timm torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [14]:
import torchvision.transforms as transforms
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
import timm
import wandb

In [15]:
data_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Adjust size as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet statistics
])
# Training set
train_dataset = torchvision.datasets.ImageFolder(
    root= '/content/Diabetic retinopathy',
    transform=data_transform,
)

# Testing set
test_dataset = torchvision.datasets.ImageFolder(
    root='/content/Renamed Images/',
    transform=data_transform,
)

# Create data loaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


In [8]:
pip install einops thop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00


In [9]:
import math
import torch
import torch.nn as nn
import numpy as np
from thop import profile
from einops import rearrange
from einops.layers.torch import Rearrange, Reduce
from timm.models.layers import trunc_normal_, DropPath

class WMSA(nn.Module):
    """ Self-attention module in Swin Transformer
    """

    def __init__(self, input_dim, output_dim, head_dim, window_size, type):
        super(WMSA, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.head_dim = head_dim
        self.scale = self.head_dim ** -0.5
        self.n_heads = input_dim//head_dim
        self.window_size = window_size
        self.type=type
        self.embedding_layer = nn.Linear(self.input_dim, 3*self.input_dim, bias=True)

        # TODO recover
        # self.relative_position_params = nn.Parameter(torch.zeros(self.n_heads, 2 * window_size - 1, 2 * window_size -1))
        self.relative_position_params = nn.Parameter(torch.zeros((2 * window_size - 1)*(2 * window_size -1), self.n_heads))

        self.linear = nn.Linear(self.input_dim, self.output_dim)

        trunc_normal_(self.relative_position_params, std=.02)
        self.relative_position_params = torch.nn.Parameter(self.relative_position_params.view(2*window_size-1, 2*window_size-1, self.n_heads).transpose(1,2).transpose(0,1))

    def generate_mask(self, w, p, shift):
        """ generating the mask of SW-MSA
        Args:
            shift: shift parameters in CyclicShift.
        Returns:
            attn_mask: should be (1 1 w p p),
        """
        # supporting sqaure.
        attn_mask = torch.zeros(w, w, p, p, p, p, dtype=torch.bool, device=self.relative_position_params.device)
        if self.type == 'W':
            return attn_mask

        s = p - shift
        attn_mask[-1, :, :s, :, s:, :] = True
        attn_mask[-1, :, s:, :, :s, :] = True
        attn_mask[:, -1, :, :s, :, s:] = True
        attn_mask[:, -1, :, s:, :, :s] = True
        attn_mask = rearrange(attn_mask, 'w1 w2 p1 p2 p3 p4 -> 1 1 (w1 w2) (p1 p2) (p3 p4)')
        return attn_mask

    def forward(self, x):
        """ Forward pass of Window Multi-head Self-attention module.
        Args:
            x: input tensor with shape of [b h w c];
            attn_mask: attention mask, fill -inf where the value is True;
        Returns:
            output: tensor shape [b h w c]
        """
        if self.type!='W': x = torch.roll(x, shifts=(-(self.window_size//2), -(self.window_size//2)), dims=(1,2))
        x = rearrange(x, 'b (w1 p1) (w2 p2) c -> b w1 w2 p1 p2 c', p1=self.window_size, p2=self.window_size)
        h_windows = x.size(1)
        w_windows = x.size(2)
        # sqaure validation
        assert h_windows == w_windows

        x = rearrange(x, 'b w1 w2 p1 p2 c -> b (w1 w2) (p1 p2) c', p1=self.window_size, p2=self.window_size)
        qkv = self.embedding_layer(x)
        q, k, v = rearrange(qkv, 'b nw np (threeh c) -> threeh b nw np c', c=self.head_dim).chunk(3, dim=0)
        sim = torch.einsum('hbwpc,hbwqc->hbwpq', q, k) * self.scale
        # Adding learnable relative embedding
        sim = sim + rearrange(self.relative_embedding(), 'h p q -> h 1 1 p q')
        # Using Attn Mask to distinguish different subwindows.
        if self.type != 'W':
            attn_mask = self.generate_mask(h_windows, self.window_size, shift=self.window_size//2)
            sim = sim.masked_fill_(attn_mask, float("-inf"))

        probs = nn.functional.softmax(sim, dim=-1)
        output = torch.einsum('hbwij,hbwjc->hbwic', probs, v)
        output = rearrange(output, 'h b w p c -> b w p (h c)')
        output = self.linear(output)
        output = rearrange(output, 'b (w1 w2) (p1 p2) c -> b (w1 p1) (w2 p2) c', w1=h_windows, p1=self.window_size)

        if self.type!='W': output = torch.roll(output, shifts=(self.window_size//2, self.window_size//2), dims=(1,2))
        return output

    def relative_embedding(self):
        cord = torch.tensor(np.array([[i, j] for i in range(self.window_size) for j in range(self.window_size)]))
        relation = cord[:, None, :] - cord[None, :, :] + self.window_size -1
        # negative is allowed
        return self.relative_position_params[:, relation[:,:,0], relation[:,:,1]]

class Block(nn.Module):
    def __init__(self, input_dim, output_dim, head_dim, window_size, drop_path, type='W', input_resolution=None):
        """ SwinTransformer Block
        """
        super(Block, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        assert type in ['W', 'SW']
        self.type = type
        if input_resolution <= window_size:
            self.type = 'W'

        print("Block Initial Type: {}, drop_path_rate:{:.6f}".format(self.type, drop_path))
        self.ln1 = nn.LayerNorm(input_dim)
        self.msa = WMSA(input_dim, input_dim, head_dim, window_size, self.type)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.ln2 = nn.LayerNorm(input_dim)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 4 * input_dim),
            nn.GELU(),
            nn.Linear(4 * input_dim, output_dim),
        )

    def forward(self, x):
        x = x + self.drop_path(self.msa(self.ln1(x)))
        x = x + self.drop_path(self.mlp(self.ln2(x)))
        return x

class SwinTransformer(nn.Module):
    """ Implementation of Swin Transformer https://arxiv.org/abs/2103.14030
    In this Implementation, the standard shape of data is (b h w c), which is a similar protocal as cnn.
    """
    #TODO make layers using configs
    def __init__(self, num_classes, config=[2,2,6,2], dim=96, drop_path_rate=0.2, input_resolution=224):
        super(SwinTransformer, self).__init__()
        self.config = config
        self.dim = dim
        self.head_dim = 32
        self.window_size = 7
        # self.patch_partition = Rearrange('b c (h1 sub_h) (w1 sub_w) -> b h1 w1 (c sub_h sub_w)', sub_h=4, sub_w=4)

        # drop path rate for each layer
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(config))]

        begin = 0
        self.stage1 = [nn.Conv2d(3, dim, kernel_size=4, stride=4),
                       Rearrange('b c h w -> b h w c'),
                       nn.LayerNorm(dim),] + \
                      [Block(dim, dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution//4)
                      for i in range(config[0])]
        begin += config[0]
        self.stage2 = [Rearrange('b (h neih) (w neiw) c -> b h w (neiw neih c)', neih=2, neiw=2),
                       nn.LayerNorm(4*dim), nn.Linear(4*dim, 2*dim, bias=False),] + \
                      [Block(2*dim, 2*dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution//8)
                      for i in range(config[1])]
        begin += config[1]
        self.stage3 = [Rearrange('b (h neih) (w neiw) c -> b h w (neiw neih c)', neih=2, neiw=2),
                       nn.LayerNorm(8*dim), nn.Linear(8*dim, 4*dim, bias=False),] + \
                      [Block(4*dim, 4*dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW',input_resolution//16)
                      for i in range(config[2])]
        begin += config[2]
        self.stage4 = [Rearrange('b (h neih) (w neiw) c -> b h w (neiw neih c)', neih=2, neiw=2),
                       nn.LayerNorm(16*dim), nn.Linear(16*dim, 8*dim, bias=False),] + \
                      [Block(8*dim, 8*dim, self.head_dim, self.window_size, dpr[i+begin], 'W' if not i%2 else 'SW', input_resolution//32)
                      for i in range(config[3])]

        self.stage1 = nn.Sequential(*self.stage1)
        self.stage2 = nn.Sequential(*self.stage2)
        self.stage3 = nn.Sequential(*self.stage3)
        self.stage4 = nn.Sequential(*self.stage4)

        self.norm_last = nn.LayerNorm(dim * 8)
        self.mean_pool = Reduce('b h w c -> b c', reduction='mean')
        self.classifier = nn.Linear(8*dim, num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.norm_last(x)

        x = self.mean_pool(x)
        x = self.classifier(x)
        return x

def Swin_T(num_classes, config=[2,2,6,2], dim=96, **kwargs):
    return SwinTransformer(num_classes, config=config, dim=dim, **kwargs)

def Swin_S(num_classes, config=[2,2,18,2], dim=96, **kwargs):
    return SwinTransformer(num_classes, config=config, dim=dim, **kwargs)

def Swin_B(num_classes, config=[2,2,18,2], dim=128, **kwargs):
    return SwinTransformer(num_classes, config=config, dim=dim, **kwargs)

def Swin_L(num_classes, config=[2,2,18,2], dim=192, **kwargs):
    return SwinTransformer(num_classes, config=config, dim=dim, **kwargs)

# if __name__ == '__main__':
#     test_model = Swin_T(1000).cuda()
#     n_parameters = sum(p.numel() for p in test_model.parameters() if p.requires_grad)
#     print(test_model)
#     dummy_input = torch.rand(3,3,224,224).cuda()
#     output = test_model(dummy_input)
#     print(output.size())
#     # flops, params = profile(test_model, inputs=(dummy_input, ))
#     # print(params)
#     # print(flops)
#     print(n_parameters)

In [19]:
class SwinTransformerClassifier(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        self.model = Swin_B(num_classes=5)
    def forward(self, x):
        return self.model(x)



In [17]:
pip install wandb

In [20]:
def train_step(model, optimizer, criterion, train_loader, device):
    model.train()
    total_loss = 0
    correct = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()
    train_loss = total_loss / len(train_loader)
    train_acc = correct / len(train_loader.dataset)
    wandb.log({"train_loss": train_loss, "train_acc": train_acc})
    return train_loss, train_acc

def val_step(model, criterion, val_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()
    val_loss = total_loss / len(val_loader)
    val_acc = correct / len(val_loader.dataset)
    wandb.log({"val_loss": val_loss, "val_acc": val_acc})
    return val_loss, val_acc

# Hyperparameters
num_epochs = 10
learning_rate = 0.001
num_classes = 5  # Adjust based on your severity levels

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize model, optimizer, loss
model = SwinTransformerClassifier(num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# Initialize Wandb run
wandb.init(project="diabetic_retinopathy_classification")


Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.008696
Block Initial Type: W, drop_path_rate:0.017391
Block Initial Type: SW, drop_path_rate:0.026087
Block Initial Type: W, drop_path_rate:0.034783
Block Initial Type: SW, drop_path_rate:0.043478
Block Initial Type: W, drop_path_rate:0.052174
Block Initial Type: SW, drop_path_rate:0.060870
Block Initial Type: W, drop_path_rate:0.069565
Block Initial Type: SW, drop_path_rate:0.078261
Block Initial Type: W, drop_path_rate:0.086957
Block Initial Type: SW, drop_path_rate:0.095652
Block Initial Type: W, drop_path_rate:0.104348
Block Initial Type: SW, drop_path_rate:0.113043
Block Initial Type: W, drop_path_rate:0.121739
Block Initial Type: SW, drop_path_rate:0.130435
Block Initial Type: W, drop_path_rate:0.139130
Block Initial Type: SW, drop_path_rate:0.147826
Block Initial Type: W, drop_path_rate:0.156522
Block Initial Type: SW, drop_path_rate:0.165217
Block Initial Type: W, drop_path_rate:0.173913
Blo

In [21]:
# Training and validation loop
for epoch in range(num_epochs):
    print(f"Epoch: {epoch+1}/{num_epochs}")

    # Training step
    train_loss, train_acc = train_step(model, optimizer, criterion, train_dataloader, device)

    # Validation step
    val_loss, val_acc = val_step(model, criterion, test_dataloader, device)

    # Print and log results
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# Save the model
torch.save(model.state_dict(), "model.pt")
print("Model saved!")

Epoch: 1/10
Train Loss: 1.6552, Train Acc: 0.4514
Val Loss: 2.7308, Val Acc: 0.1235
Epoch: 2/10
Train Loss: 1.3440, Train Acc: 0.4776
Val Loss: 2.3942, Val Acc: 0.0678
Epoch: 3/10


KeyboardInterrupt: 